# Compare to:
## 1. An empirical study on program failures of deep learning jobs, Zhang et al, ICSE20

### Label automatically notebooks that are DL

based on what libraries they use

In [34]:
import pandas as pd
import utils.config as config
import utils.util as util
import numpy as np

# manually labeled
df_mlerr_labels = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_processed.xlsx'))
df_mlerr_labels_sum = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_sum.xlsx'))

df_err_g = pd.read_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'))
df_err_k = pd.read_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'))

df_err_g.drop_duplicates(subset=['eid'], keep='first', inplace = True)
df_err_k.drop_duplicates(subset=['eid'], keep='first', inplace = True)

def get_DL(df_mlerr_labels, save_path = None):
    df_mlerr_labels = pd.merge(df_mlerr_labels, 
                               pd.concat([df_err_g[["eid","lib_alias"]], df_err_k[["eid","lib_alias"]]], ignore_index=True), 
                               on="eid", how="left")
    df_mlerr_labels["is_DLnb"] = df_mlerr_labels.lib_alias.apply(util.lib_alias_isDL)
    print(sum(df_mlerr_labels["is_DLnb"])/df_mlerr_labels.shape[0])
    print(df_mlerr_labels[df_mlerr_labels.is_DLnb==True].fname.nunique()/df_mlerr_labels.fname.nunique())

    df_mlerr_labels = df_mlerr_labels[df_mlerr_labels.is_DLnb==True]
    df_mlerr_labels = df_mlerr_labels.drop(['is_DLnb', 'lib_alias'], axis=1)
    if save_path:
        df_mlerr_labels.to_excel(save_path, index=False, engine='xlsxwriter')
    
get_DL(df_mlerr_labels, save_path = config.path_default.joinpath("Manual_labeing/cluster_sampled_labeled_DL.xlsx"))
get_DL(df_mlerr_labels_sum, save_path = config.path_default.joinpath("Manual_labeing/cluster_sampled_labeled_DL_sum.xlsx"))

0.4584450402144772
0.4599447513812155
0.4584450402144772
0.4599447513812155


### Map our labeling to theirs: 

    Execution environment: 
        label_root_cause == config.label_root_cause["environment"]
    Data (dueing data preocessing, data integrity is compromised. i.e., corrupt data, unexpected encoding): 
        label_refined_exp_type == ["jsondecodeerror", "unsupported file type (read file)", "incompleteparseerror"]
        ename=="unicodedecodeerror"
    DL specific: 
        label_if_ML_bug == config.label_if_ML_bug["ML bug"]
    General code error: 
        label_if_ML_bug == config.label_if_ML_bug["python bug"]
    

In [8]:
import pandas as pd
import utils.config as config
import utils.util as util
import numpy as np

df_mlerr_labels_DL = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_DL.xlsx'))
df_mlerr_labels_sum_DL = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_DL_sum.xlsx'))

In [9]:
df_mlerr_labels_DL["Comp1_DLJobs"] = -1
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where(df_mlerr_labels_DL['label_root_cause'].isin(config.label_root_cause["environment"]),
                                              "Execution environment",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where((df_mlerr_labels_DL['label_refined_exp_type'].isin(["jsondecodeerror", "unsupported file type (read file)", "incompleteparseerror"])|(df_mlerr_labels_DL['ename']=="unicodedecodeerror"))&(df_mlerr_labels_DL["Comp1_DLJobs"]=="-1"),
                                              "Data",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where(df_mlerr_labels_DL['label_if_ML_bug'].isin(config.label_if_ML_bug["ML bug"])&(df_mlerr_labels_DL["Comp1_DLJobs"]=="-1"),
                                              "DL specific",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where(df_mlerr_labels_DL['label_if_ML_bug'].isin(config.label_if_ML_bug["python bug"])&(df_mlerr_labels_DL["Comp1_DLJobs"]=="-1"),
                                              "General code error",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_sum_DL = pd.merge(df_mlerr_labels_sum_DL, 
                               df_mlerr_labels_DL[["eid","Comp1_DLJobs"]], 
                               on="eid", how="left")

In [10]:
df_mlerr_labels_DL["Comp1_DLJobs"].value_counts()/df_mlerr_labels_DL.shape[0]

Comp1_DLJobs
DL specific              0.599415
Execution environment    0.198830
General code error       0.192982
Data                     0.008772
Name: count, dtype: float64

DL specific:

    GPU out of memory/CPU out of memory:
    Framework API misuse:
    Tensor Mismatch
    Loss NaN: - 
    
Execution environment:

    Path not found:
    Library not found:
    Permission denied:
    
General code error:

    Illegal argument:
    Type mismatch:
    ...
    
Data:

    Corrupt data
    Unexpected encoding